## Use new PyRosetta4 bindings

Reads a single pdb file, given a list of residues that are "active site residues" and measures the CA and CB distances for all pairings.

Then prints these out for use in the upweighted.alignment.grishin.cst files

In [ ]:
import rosetta
import pyrosetta
from rosetta import *
pyrosetta.init('-ignore_unrecognized_res T -ignore_waters T -prevent_repacking T')

In [ ]:
pdbcode = '1w6y'  # this is just used for storing the pdb code 
crystalfile = '../modelsysteminputs/1w6y.docked_input.pdb' #use input pdb/or crystal structure
catres = [14,38,101]   #define the catalytic residues, by a list or from the REMARKS header

In [ ]:
#crystal = pose_from_file(crystalfile)
crystal = pyrosetta.pose_from_file(crystalfile)

In [ ]:
#  i are the catalytic residues in the xtal struct. This will be different than the    
#  pose numbering. So loop over each cat res, and get the correct pose number
#

# This assumes chain A is the correct chain, pull the CA and CB xyz vectors from the
# pose, then calculated the distances.
import pandas as pd

listofseries = []

for i in catres:    
    
    poseresi = crystal.pdb_info().pdb2pose('A',i)
    xyzi_CA = crystal.residue(poseresi).atom('CA').xyz() 
    xyzi_CB = crystal.residue(poseresi).atom('CB').xyz()    

    for j in catres:
        if j > i:
            poseresj = crystal.pdb_info().pdb2pose('A',j)
            xyzj_CA = crystal.residue(poseresj).atom('CA').xyz() 
            xyzj_CB = crystal.residue(poseresj).atom('CB').xyz()    
            
            ##
            ## Calc distances
            ##
            CA_CA_d = xyzi_CA.distance(xyzj_CA)
            CB_CB_d = xyzi_CB.distance(xyzj_CB)
            resiCA_resjCB = xyzi_CA.distance( xyzj_CB )
            resiCB_resjCA = xyzj_CA.distance( xyzi_CB )
            
            ## Now store the data
            mydict = { 'name' : pdbcode,
                'crystalresi': int(i),
                    'crystalresj':int(j),
                    'CA_distance': CA_CA_d,
                    'CB_distance': CB_CB_d,
                    'resi_CA_to_resj_CB_distance': resiCA_resjCB,
                    'resj_CA_to_resi_CB_distance': resiCB_resjCA,                      
                    
                     }
                
            listofseries.append(pd.Series(mydict))
df = pd.DataFrame(listofseries)

In [14]:
#print out your data
df

,CA_distance,CB_distance,crystalresi,crystalresj,name,resi_CA_to_resj_CB_distance,resj_CA_to_resi_CB_distance
0,14.531603,12.378127,14,38,1w6y,13.427255,13.395572
1,10.690744,7.865900,14,101,1w6y,9.253739,9.302595
2,8.361522,7.958868,38,101,1w6y,8.213799,8.368847


In [15]:
#store the new lines to add here
addcstlines = []
for i in catres:
    for j in catres:
        if j > i:        
                # for each catalytic pair, get the CA-CA, CB-CB and both cross distances CA-CB CB-CA
                addcstlines.append("AtomPair CA %s CA %s SCALARWEIGHTEDFUNC %s HARMONIC %.2f %s \n" %(i,j,1000,df[ (df.crystalresi == i) & (df.crystalresj == j )].CA_distance.values[0],0.5))
                addcstlines.append("AtomPair CB %s CB %s SCALARWEIGHTEDFUNC %s HARMONIC %.2f %s \n" %(i,j,1000,df[ (df.crystalresi == i) & (df.crystalresj == j )].CB_distance.values[0],0.5))
                addcstlines.append("AtomPair CA %s CB %s SCALARWEIGHTEDFUNC %s HARMONIC %.2f %s \n" %(i,j,1000,df[ (df.crystalresi == i) & (df.crystalresj == j )].resi_CA_to_resj_CB_distance.values[0],0.5))
                addcstlines.append("AtomPair CB %s CA %s SCALARWEIGHTEDFUNC %s HARMONIC %.2f %s \n" %(i,j,1000,df[ (df.crystalresi == i) & (df.crystalresj == j )].resj_CA_to_resi_CB_distance.values[0],0.5))

# see your constraints.
for i in addcstlines:
    print i

AtomPair CA 14 CA 38 SCALARWEIGHTEDFUNC 1000 HARMONIC 14.53 0.5 

AtomPair CB 14 CB 38 SCALARWEIGHTEDFUNC 1000 HARMONIC 12.38 0.5 

AtomPair CA 14 CB 38 SCALARWEIGHTEDFUNC 1000 HARMONIC 13.43 0.5 

AtomPair CB 14 CA 38 SCALARWEIGHTEDFUNC 1000 HARMONIC 13.40 0.5 

AtomPair CA 14 CA 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 10.69 0.5 

AtomPair CB 14 CB 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 7.87 0.5 

AtomPair CA 14 CB 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 9.25 0.5 

AtomPair CB 14 CA 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 9.30 0.5 

AtomPair CA 38 CA 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 8.36 0.5 

AtomPair CB 38 CB 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 7.96 0.5 

AtomPair CA 38 CB 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 8.21 0.5 

AtomPair CB 38 CA 101 SCALARWEIGHTEDFUNC 1000 HARMONIC 8.37 0.5 

